# CSE 572: Lab 10

In this lab, you will practice implementing techniques for model selection including cross validation and grid search.

To execute and make changes to this notebook, click File > Save a copy to save your own version in your Google Drive or Github. Read the step-by-step instructions below carefully. To execute the code, click on each cell below and press the SHIFT-ENTER keys simultaneously or by clicking the Play button. 

When you finish executing all code/exercises, save your notebook then download a copy (.ipynb file). Submit the following **three** things:
1. a link to your Colab notebook,
2. the .ipynb file, and
3. a pdf of the executed notebook on Canvas.

To generate a pdf of the notebook, click File > Print > Save as PDF.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

# Set the random seed for reproducibility
seed = 0
np.random.seed(0)

### Load the iris dataset

In [2]:
data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)
data.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'class']

data.sample(5, random_state=seed)

,sepal length,sepal width,petal length,petal width,class
114,5.8,2.8,5.1,2.4,Iris-virginica
62,6.0,2.2,4.0,1.0,Iris-versicolor
33,5.5,4.2,1.4,0.2,Iris-setosa
107,7.3,2.9,6.3,1.8,Iris-virginica
7,5.0,3.4,1.5,0.2,Iris-setosa


In [3]:
data.shape

(150, 5)

Standardize the data by subtracting the feature-wise mean and dividing by the feature-wise standard deviation for each sample.

In [4]:
# YOUR CODE HERE
for x in data.columns:
  if not x == 'class':
    mean, std = data[x].mean(), data[x].std()
    data[x] = (data[x] - mean) / std

In [5]:
data.sample(5, random_state=seed)

,sepal length,sepal width,petal length,petal width,class
114,-0.052331,-0.585801,0.760212,1.574155,Iris-virginica
62,0.189196,-1.969583,0.136778,-0.260321,Iris-versicolor
33,-0.414621,2.643024,-1.336794,-1.308593,Iris-setosa
107,1.759119,-0.355171,1.440322,0.787951,Iris-virginica
7,-1.018437,0.797981,-1.280118,-1.308593,Iris-setosa


### k-fold Cross validation

We will use 5-fold cross validation to train and evaluate our classifier. We will not do any model selection/hyperparameter tuning in this step, so we need to split our data into a training and test set.

To split the data into 5 folds we will shuffle the rows and then split them into $k$ equal groups.

In [6]:
k = 5

# Note: np.split raises error if indices_or_sections is 
# an integer and doesn't result in equal size splits
folds = np.split(data.sample(frac=1, random_state=seed), indices_or_sections=k)

Use a for loop to print the number of samples and number of samples from each class in each fold.

In [28]:
# YOUR CODE HERE
for i, fold in enumerate(folds):
  print('Fold ' + str(i+1) + ': (' + str(fold.shape[0]) + ' samples)')
  print(fold['class'].value_counts())
  print()

Fold 1: (30 samples)
Iris-versicolor    13
Iris-setosa        11
Iris-virginica      6
Name: class, dtype: int64

Fold 2: (30 samples)
Iris-virginica     15
Iris-versicolor    10
Iris-setosa         5
Name: class, dtype: int64

Fold 3: (30 samples)
Iris-versicolor    10
Iris-setosa        10
Iris-virginica     10
Name: class, dtype: int64

Fold 4: (30 samples)
Iris-setosa        14
Iris-virginica     10
Iris-versicolor     6
Name: class, dtype: int64

Fold 5: (30 samples)
Iris-versicolor    11
Iris-setosa        10
Iris-virginica      9
Name: class, dtype: int64



### Train a k Nearest Neighbors classifier 

We will use the [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) in sklearn for our classification model. Use cross validation to train and evaluate the model. Set hyperparameters to `n_neighbors=5`, `metric='l2'`, and `weights='uniform'`.

Implement a for loop to iterate through each fold, training a new kNN model each iteration with one fold assigned to validation and the remaining folds assigned to training. Compute the validation accuracy for each iteration and append it to the `accuracies` list.

In [30]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

accuracies = []
# YOUR CODE HERE

for i, fold in enumerate(folds):
  train = pd.concat(folds[:i] + folds[i+1:])
  validation = fold
  clf = KNeighborsClassifier(n_neighbors=5, metric='l2', weights='uniform')
  X_train, y_train = train.drop('class', axis=1), train['class']
  clf.fit(X_train, y_train)
  X_val, y_val = validation.drop('class', axis=1), validation['class']
  y_val_pred = clf.predict(X_val)
  accuracies.append(accuracy_score(y_val, y_val_pred))

Print the mean and standard deviation of the accuracy from cross validation (across all $k$ folds).

In [9]:
print('Mean accuracy: {:.2f}'.format(np.mean(accuracies)))
print('Standard deviation of accuracy: {:.2f}'.format(np.std(accuracies)))

Mean accuracy: 0.95
Standard deviation of accuracy: 0.06


**Question 1: If you increased the number of folds, do you expect the standard deviation of the accuracy across $k$ folds to increase or decrease? Why?**

**Answer:**

Increasing the number of folds will increase the standard deviation of the accuracy. This is because, increasing k leads to a smaller number of samples in the validation set.

### Hyperparameter selection using cross validation and grid search

In this exercise, we will use the [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) again but this time we will perform hyperparameter selection using k-fold cross validation and Grid Search. 

We have three model choices (hyperparameters) for our kNN model:
- Number of neighbors ($k$ or `n_neighbors`). We will consider all integer values $k \in [1,10]$.
- Whether to treat all neighbors equally when taking majority vote, or weight them according to their distance from the query point (`weights='uniform'` or `weights='distance'`).
- The distance metric for computing distance between query point and neighbors (`metric` argument). We will consider three options for `metric`: `'l1'`, `'l2'`, and `'cosine'`.

**Question 2: How many total combinations of the above hyperparameter choices are there?**

**Answer:**

We have 60 (10 x 2 x 3) total combinations of hyperparameter choices.

Instead of implementing cross validation manually as we did in the previous example, we will use the [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) class in sklearn to perform grid search and cross validation simultaneously. 

First, we will split the data into a training (70\%) and test (30\%) test.

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[data.columns[:-1]], 
                                                    data['class'], 
                                                    test_size=0.3, 
                                                    random_state=seed)

We will then use the training set for cross validation and grid search to select the optimal hyperparameter settings.

Next, we define the values for grid search using a dictionary in which the keys are the parameter names to be passed to the model function and each corresponding value is a list of possible values to try in grid search.

In [11]:
param_grid = {'n_neighbors': list(range(1, 11)), 
              'weights': ['uniform', 'distance'],
              'metric': ['l1', 'l2', 'cosine']
             }

Next, we instantiate a kNeighborsClassifier but do not specify the hyperparameter settings yet.

In [12]:
knn = KNeighborsClassifier()

We can then pass this classifier and our parameter grid to a new GridSearchCV object and fit the GridSearchCV using our training data.

In [13]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(knn, param_grid)

clf.fit(X_train, y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'metric': ['l1', 'l2', 'cosine'],
                         'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'weights': ['uniform', 'distance']})

The cross validation results are stored as an attribute of the GridSearchCV object as a dictionary with keys as column headers and values as columns, that can be imported into a pandas DataFrame.

In [14]:
cv_results = pd.DataFrame(clf.cv_results_)

cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_metric,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004355,0.000309,0.005540,0.001140,l1,1,uniform,"{'metric': 'l1', 'n_neighbors': 1, 'weights': ...",0.857143,0.904762,1.000000,0.904762,0.952381,0.923810,0.048562,37
1,0.002973,0.000513,0.002835,0.000190,l1,1,distance,"{'metric': 'l1', 'n_neighbors': 1, 'weights': ...",0.857143,0.904762,1.000000,0.904762,0.952381,0.923810,0.048562,37
2,0.002690,0.000448,0.003639,0.000308,l1,2,uniform,"{'metric': 'l1', 'n_neighbors': 2, 'weights': ...",0.857143,0.904762,1.000000,0.857143,0.952381,0.914286,0.055533,44
3,0.002448,0.000161,0.002629,0.000308,l1,2,distance,"{'metric': 'l1', 'n_neighbors': 2, 'weights': ...",0.857143,0.904762,1.000000,0.904762,0.952381,0.923810,0.048562,37
4,0.006771,0.008067,0.004564,0.001496,l1,3,uniform,"{'metric': 'l1', 'n_neighbors': 3, 'weights': ...",0.904762,1.000000,1.000000,0.857143,0.952381,0.942857,0.055533,10
5,0.004617,0.003549,0.008473,0.004961,l1,3,distance,"{'metric': 'l1', 'n_neighbors': 3, 'weights': ...",0.904762,1.000000,1.000000,0.904762,0.952381,0.952381,0.042592,2
6,0.013395,0.002733,0.011958,0.006875,l1,4,uniform,"{'metric': 'l1', 'n_neighbors': 4, 'weights': ...",0.857143,1.000000,1.000000,0.904762,1.000000,0.952381,0.060234,2
7,0.006127,0.003638,0.004482,0.000806,l1,4,distance,"{'metric': 'l1', 'n_neighbors': 4, 'weights': ...",0.857143,1.000000,1.000000,0.904762,0.952381,0.942857,0.055533,10
8,0.003549,0.001921,0.005444,0.003957,l1,5,uniform,"{'metric': 'l1', 'n_neighbors': 5, 'weights': ...",0.809524,1.000000,1.000000,0.904762,0.952381,0.933333,0.071270,27
9,0.002562,0.000460,0.002728,0.000264,l1,5,distance,"{'metric': 'l1', 'n_neighbors': 5, 'weights': ...",0.809524,1.000000,1.000000,0.904762,0.952381,0.933333,0.071270,27


Look at the [GridSearchCV documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) to read about the other attributes stored after fitting. Print the value of the attribute that gives the parameter settings for the best results on the hold out data.

In [20]:
# YOUR CODE HERE
clf.best_params_

{'metric': 'l1', 'n_neighbors': 6, 'weights': 'uniform'}

Train a new kNN classifier using the hyperparameter settings that were found to give the best results on the hold out data from GridSearchCV (the values printed in the last cell). Train it on the full training set.

In [23]:
# YOUR CODE HERE
clf = KNeighborsClassifier(n_neighbors=6, metric='l1', weights='uniform')
clf.fit(X_train, y_train)

KNeighborsClassifier(metric='l1', n_neighbors=6)

Apply the trained classifier to the test dataset and print the test accuracy.

In [24]:
# YOUR CODE HERE
accuracy_score(y_test, clf.predict(X_test))

0.9777777777777777